In [4]:
import gpn.model
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from sklearn.preprocessing import StandardScaler
import torch
from transformers import AutoModel, AutoModelForMaskedLM, AutoTokenizer
import h5py
import sys
from tqdm import tqdm
import numpy as np
sys.path.append('../')
import utils
import os
from tqdm import tqdm
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

model = AutoModel.from_pretrained("/home/ztang/multitask_RNA/model/GPN_human/checkpoint-2000000").to('cuda')
tokenizer = AutoTokenizer.from_pretrained("/home/ztang/multitask_RNA/model/GPN_human/checkpoint-2000000")

celltype = 'K562'
file = h5py.File('/home/ztang/multitask_RNA/data/CAGI/'+celltype+'/onehot.h5','r')

Some weights of the model checkpoint at /home/ztang/multitask_RNA/model/GPN_human/checkpoint-2000000 were not used when initializing ConvNetModel: ['cls.decoder.2.weight', 'cls.decoder.3.weight', 'cls.decoder.0.weight', 'cls.decoder.2.bias', 'cls.decoder.3.bias', 'cls.decoder.0.bias']
- This IS expected if you are initializing ConvNetModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ConvNetModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [5]:
gpn_output = h5py.File('../../data/CAGI_embed/'+celltype+'/gpn.h5','w')
N,L,A = file['alt'].shape
batch_size = 64

for set in ['alt','ref']:
    embed = []
    for i in tqdm(range(0,N,batch_size)):
        onehot_seq = file[set][i:i+batch_size]
        seq = utils.onehot_to_seq(onehot_seq)
        token = tokenizer.batch_encode_plus(seq)['input_ids']
        output = model(torch.tensor(token).to('cuda')).last_hidden_state.cpu().detach().numpy()
        embed.extend(output)
    gpn_output.create_dataset(name=set,data = np.array(embed))


100%|██████████| 23/23 [00:01<00:00, 17.32it/s]


In [6]:
gpn_output.close()